In [129]:
import pandas as pd
import numpy as np

# Input

test_input_raw= \
"""7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7"""

with open("./inputs/d4p1.txt", "r") as f:
    input_raw = f.read()
use_test = False
input = test_input_raw.splitlines() if use_test else input_raw.splitlines()

# Get draws first and remove from input
draws = [int(x) for x in input[0].split(",")]
input = input[2:]
boards = [input[i:i+5] for i in range(0, len(input), 6)]
boards = [[x.split() for x in board] for board in boards]

boards = np.array(boards, dtype=int)


### Approach for Part 1

Store the boards themselves as (5,5) ndArrays. Store an answer ndArray for each board wher dType is int.

### Checking for a Win
With the answers board we can check all the rows, columnns, and diagnols for a sum or 3 (1 + 1 + 1).

#### Diaganols
The diaganols can be checked using `np.trace`, along with a `np.rot90` followed by `np.trace` to check the other diaganol.
For the row and column win conditions we can use `np.sum` along the specified axis to see if there are any `sums` that indicate a win.

In [112]:
# The core win checking logic that is used throughout

def check_for_win(answers, diaganols=False):
    answer_mask = None
    win_dim = answers.shape[0]

    if diaganols:
        # Check first diaganol
        if np.trace(answers) == win_dim:
            answer_mask = np.eye(win_dim, dtype=int)

        # Check second diaganol
        if np.trace(np.rot90(answers)) == win_dim:
            answer_mask = np.rot90(np.eye(win_dim, dtype=int))

    # Check rows
    row_sum = answers.sum(axis=1)
    if win_dim in row_sum:
        answer_mask = np.zeros(answers.shape, dtype=int)
        answer_mask[row_sum == win_dim, :] = 1

    # Check columns
    col_sum = answers.sum(axis=0)
    if win_dim in answers.sum(axis=0):
        answer_mask = np.zeros(answers.shape)
        answer_mask[:, col_sum == win_dim] = 1

    if answer_mask is not None:
        return (True, answer_mask)
    else:
        return (False, np.zeros(win_dim))
    

In [ ]:
# Tests for win check

import unittest
import numpy.testing as npt

class TestCheckWin(unittest.TestCase):
    def test_empty(self):
        s = (5,5)
        test_board = np.zeros(s, dtype=int)
        result = check_for_win(test_board)
        self.assertEqual(result[0], False)
        self.assertEqual(result[1].any(), False)

    def test_diaganol(self):
        test_board = np.eye(5, dtype=int)
        result = check_for_win(test_board)
        self.assertEqual(result[0], True)
        npt.assert_array_equal(result[1], np.eye(5))
        
    def test_rot_diaganol(self):
        test_board = np.rot90(np.eye(5, dtype=int))
        result = check_for_win(test_board)
        self.assertEqual(result[0], True)
        npt.assert_array_equal(result[1], np.rot90(np.eye(5)))

    def test_row(self):
        test_board = np.zeros((5,5), dtype=int)
        test_board[3,:] = 1
        answer = test_board
        result = check_for_win(test_board)
        self.assertEqual(result[0], True)
        npt.assert_array_equal(result[1], answer) 

    def test_col(self):
        test_board = np.zeros((5,5), dtype=int)
        test_board[:,3] = 1
        answer = test_board
        result = check_for_win(test_board)
        self.assertEqual(result[0], True)
        npt.assert_array_equal(result[1], answer) 

unittest.main(argv=[''], exit=False)


In [ ]:
# The sloppy original solution -- worked but was hacked together for speed
# Set up for part two (last board win) currently

answers = np.zeros(boards.shape, dtype=int)
win_set = set()
for (draw_num, draw) in enumerate(draws):

    print(f"draw_num: {draw_num} draw: {draw}")

    for (i,board) in enumerate(boards):
        # Check for hits and update answers board
        hit = (board == draw)
        answers[i][hit] = 1

        # Now check for a win
        (is_win, win_mask) = check_for_win(answers[i])

        if is_win:
            win_set.add(i)
            if len(win_set) == len(boards):
                print("====================")
                print("====================")
                print("====================")
                print("====================")
                print("====================")
                print("====================")
            print("=======")
            answers_invert_mask = np.where(answers[i] == 0, 1, 0)
            invert_board = answers_invert_mask * board
            invert_board_sum = np.sum(invert_board)
            print(invert_board_sum)
            print(draw)
            print(f"Final Score is: {invert_board_sum * draw}")

In [ ]:
def play_boards_first_win(draw, boards, answers):
    """Adds the draw to boards (answers) array, returns on first win"""

def play_boards_last_win(draw, boards, answers):